In [1]:
import pandas as pd
import numpy as np

# Read the Republic Data file - Plato document

    - This file is available at http://www.gutenberg.org/cache/epub/1497/pg1497.txt
    

In [6]:
def read_file(file):
    f = open(file, 'r', encoding='utf-8')
    lines = f.readlines()
    data = '\t'.join([line.strip() for line in lines])
    return data

data = read_file('republic.txt')

# Clean the Read file

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Text cleansing

    - Good article available at https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/

In [8]:
import re
import string
from nltk.corpus import stopwords
import pandas as pd

def clean_doc(doc):
    
    #replace -- with a space
    doc = doc.replace('-',' ')
    
    #replace -- with a space
    doc = doc.replace('.',' ')
    
    #split into tokens by white space
    tokens = doc.split()
    
    # prepare for char filtering
    re_punc = re.compile('[%s]'%re.escape(string.punctuation))
    
    #remove punctuation from words
    tokens = [re_punc.sub('',w) for w in tokens]
    
    #remove remaining tokens that are not alphanumerics
    tokens = [word for word in tokens if word.isalpha()]
    
    #make lower case
    tokens = [word.lower() for word in tokens]
    
    #remove stop words & words which appear less than 1 times
    stops = set(stopwords.words("english"))
    #tokens = [word for word in tokens if not word in stops and len(word)>=1]

    return tokens

tokens = clean_doc(data)
                 
print ('Number of tokens are ', len(tokens))
print ('Number of unique tokens is ', len(set(tokens)))

Number of tokens are  8578
Number of unique tokens is  1874


# Generate Sequences

    - This method takes in bunch of tokens and then converts those tokens into sequences. The idea here is that we create
    sequences of 21 tokens (words) each and then feed each sequence one at a time to the network. each word in the sequence represents a value at a given timesteps. All words in effect become the features

In [9]:
words_per_sequence = 10

def generate_sequences(size, tokens):
    sequences = list()
    size_seq = size + 1
    for i in range (0, len(tokens)-size):
        seq = tokens[i:size_seq+i]
        line = ' '.join(seq)
        sequences.append(line)
        
    print ('Total number of sequenes is ', len(sequences))
    return sequences

sequences = generate_sequences(words_per_sequence, tokens)

Total number of sequenes is  8568


# Save generated sequences in new file

In [10]:
def save_sequences(filename, sequences):
    
    file = open(filename,'w',encoding='utf-8')
    lines = '\n'.join(sequences)
    file.write(lines)
    file.close()
    
save_sequences('cleaned-republic.txt', sequences)

# Read cleaned sequences from file

In [11]:
def read_sequences(file):
    f = open(file, 'r', encoding='utf-8')
    lines = f.read()
    return lines

lines = read_sequences('cleaned-republic.txt')
lines = lines.split('\n')

# Tokenize & Create Sequences of Numbers

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np

def tokenize_documents(lines):
    tokenizer = Tokenizer(num_words=2000)
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(lines)
    return (tokenizer, sequences)

(tokenizer, sequences) = tokenize_documents(lines)
sequences = pad_sequences(sequences, maxlen=words_per_sequence, padding='pre')

In [13]:
vocab_len = len (tokenizer.word_index)+1
print ('Vocab length is ', vocab_len)
print ('Number of sequences is ', len(sequences))
maxlength = max([len(seq) for seq in sequences])
print ('Max length of all sequences is ', maxlength)

Vocab length is  1875
Number of sequences is  8568
Max length of all sequences is  10


In [14]:
from keras.utils import to_categorical

sequences = np.array(sequences)
X = sequences[:,:words_per_sequence]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocab_len)

# Configure Callbacks which would be executed during the training phase

In [15]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

def create_callbacks():
    callbacks = [
        EarlyStopping(monitor='acc', patience=5),
        ModelCheckpoint(filepath='republic-model.h5', monitor='val_loss', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    ]
    return callbacks

# Actual Language Model

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Conv1D, Dropout

def create_model():
    model = Sequential()
    
    print ('vocab len ', vocab_len)
    
    #Embedding layer has three attributes
        # input_dim - size of the vocabulary
        # output_dim - number of dimensions in which each word is embedded
        # input_length - length of each word
    model.add(Embedding(input_dim=vocab_len,output_dim=50,input_length=words_per_sequence))
    model.add(Conv1D(filters=10, kernel_size=4, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=20, kernel_size=4, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=30, kernel_size=4, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_len, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [24]:
model = create_model()
cbk = create_callbacks()
#history = model.fit(X,y, epochs=10, validation_split=0.2, batch_size=32, callbacks=cbk)
history = model.fit(X,y, epochs=10, validation_split=0.2, batch_size=32, callbacks=cbk)

vocab len  1875
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 50)            93750     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 7, 10)             2010      
_________________________________________________________________
dropout_7 (Dropout)          (None, 7, 10)             0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 4, 20)             820       
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 20)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1, 30)             2430      
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 30)             0    

ValueError: Error when checking target: expected dense_4 to have 3 dimensions, but got array with shape (8568, 1875)

# Visualize the Training Metadata (Accuracy & Loss)

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

print (history)
figure = plt.figure()

ax1 = figure.add_subplot(211)
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(labels=['Training','Validation'])

ax1 = figure.add_subplot(212)
ax1.plot(history.history['loss'])
ax1.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(labels=['Training','Validation'])

plt.show()

NameError: name 'history' is not defined

# Save the model

In [ ]:
from pickle import dump
model.save('languagemodelling-cnn-lstm-model.h5')
dump(tokenizer, open('languagemodelling-tokenizer.h5','wb'))

# Load the model

In [ ]:
from pickle import load
from keras.models import load_model

lmodel = load_model('languagemodelling-cnn-lstm-model.h5')
ltokenizer = load(open('languagemodelling-tokenizer.h5','rb'))

#load clean token file
def read_cleaned_file(file):
    f = open(file,'r', encoding='utf-8')
    lines = f.read()
    return lines

lines = read_cleaned_file('cleaned-republic.txt')
lines = lines.split('\n')

In [ ]:
# Randomly select a line

import random
linenumber = random.randint(0,200)

seed = lines[linenumber]
print ('%s'%(seed))

In [ ]:
from keras.preprocessing.sequence import pad_sequences

def generate_sequences(model, tokenizer, seed, number):
    result = list()
    
    orig_seed = seed
    
    for _ in range (number):
        
        encoder = tokenizer.texts_to_sequences([seed])[0]
        
        padded = pad_sequences([encoder], truncating='pre', maxlen=words_per_sequence)
        
        yhat = model.predict_classes(padded)
        
        output = ' '
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                output = word
                break
        print ('%s %s'%(orig_seed, output))
        seed = seed + ' ' + output
        
        result.append(output)
        
generate_sequences(model, tokenizer, seed, 20)